In [9]:
import sys
import time
from PIL import Image, ImageDraw
from utils import *
import matplotlib.pyplot as plt
import time
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.backends.cudnn as cudnn
from torchvision import datasets, transforms
from torch.autograd import Variable
%matplotlib inline
import numpy as np
import dataset
import random
import math
import json
from region_loss import RegionLoss
from models import *
import h5py
os.environ['CUDA_VISIBLE_DEVICES'] = '3'
torch.cuda.manual_seed(int(time.time()))

In [17]:
transform=transforms.Compose([
                           transforms.ToTensor(),
                           transforms.Normalize(mean = [ 0.5, 0.5, 0.5 ],std = [ 0.25, 0.25, 0.25 ]),
                       ])

In [25]:
a = 'FullChannels()'

In [26]:
model = eval(a)

In [27]:
load_net('backup/FullChannels-best.weights', model)
model.cuda()

FullChannels(
  (model): Sequential(
    (0): Sequential(
      (0): Conv2d(3, 3, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=3)
      (1): ReLU(inplace)
      (2): Conv2d(3, 32, kernel_size=(1, 1), stride=(1, 1))
      (3): ReLU(inplace)
    )
    (1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (2): Sequential(
      (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32)
      (1): ReLU(inplace)
      (2): Conv2d(32, 64, kernel_size=(1, 1), stride=(1, 1))
      (3): ReLU(inplace)
    )
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Sequential(
      (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=64)
      (1): ReLU(inplace)
      (2): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1))
      (3): ReLU(inplace)
    )
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Sequential(
      (0): Conv

In [28]:
count = 0
for k, v in model.state_dict().items():
    data=v.cpu().numpy()
    if count<16 and count%2==0:
        minimum=1.0/2**7
    else:
        minimum=1.0/2**7
    for x in np.nditer(data, op_flags=['readwrite']):
        if x[...]>1:
            x[...]=0.99
        if x[...]<-1:
            x[...]=-0.99
        x[...]=round(x[...]/minimum)*minimum
    param = torch.from_numpy(data)  
    v.copy_(param)

In [29]:
with open('test.txt', 'r') as outfile:
    lines = json.load(outfile)

In [30]:
test_infos = lines[0]

In [34]:
pred_infos = []
anchors     = model.anchors
num_anchors = model.num_anchors
anchor_step = len(anchors)/num_anchors
proposals = 0.0
total = 0.0
for i in range(len(test_infos)):
    imgpath = test_infos[i][0]
    box = test_infos[i][1]
    label = np.zeros(4)
    label[0:2] = box[2:4]
    label[2:4] = box[0:2]
    img = Image.open(imgpath).convert('RGB')
    img = img.resize((model.width, model.height))
    timg = transform(img)
    timg = timg.view(1, 3, model.height, model.width)
    output = model(timg.cuda()).data
    batch = output.size(0)
    h = output.size(2)
    w = output.size(3)
    output = output.view(batch*num_anchors, 5, h*w).transpose(0,1).contiguous().view(5, batch*num_anchors*h*w)
    grid_x = torch.linspace(0, w-1, w).repeat(h,1).repeat(batch*num_anchors, 1, 1).view(batch*num_anchors*h*w).cuda()
    grid_y = torch.linspace(0, h-1, h).repeat(w,1).t().repeat(batch*num_anchors, 1, 1).view(batch*num_anchors*h*w).cuda()
    xs = torch.sigmoid(output[0]) + grid_x
    ys = torch.sigmoid(output[1]) + grid_y

    anchor_w = torch.Tensor(anchors).view(num_anchors, anchor_step).index_select(1, torch.LongTensor([0]))
    anchor_h = torch.Tensor(anchors).view(num_anchors, anchor_step).index_select(1, torch.LongTensor([1]))
    anchor_w = anchor_w.repeat(batch, 1).repeat(1, 1, h*w).view(batch*num_anchors*h*w).cuda()
    anchor_h = anchor_h.repeat(batch, 1).repeat(1, 1, h*w).view(batch*num_anchors*h*w).cuda()
    ws = torch.exp(output[2]) * anchor_w
    hs = torch.exp(output[3]) * anchor_h
    det_confs = torch.sigmoid(output[4])
    sz_hw = h*w
    sz_hwa = sz_hw*num_anchors
    det_confs = convert2cpu(det_confs)
    xs = convert2cpu(xs)
    ys = convert2cpu(ys)
    ws = convert2cpu(ws)
    hs = convert2cpu(hs)        

    for b in range(batch):
        det_confs_inb = det_confs[b*sz_hwa:(b+1)*sz_hwa].numpy()
        xs_inb = xs[b*sz_hwa:(b+1)*sz_hwa].numpy()
        ys_inb = ys[b*sz_hwa:(b+1)*sz_hwa].numpy()
        ws_inb = ws[b*sz_hwa:(b+1)*sz_hwa].numpy()
        hs_inb = hs[b*sz_hwa:(b+1)*sz_hwa].numpy()      
        ind = np.argmax(det_confs_inb)

        bcx = xs_inb[ind]
        bcy = ys_inb[ind]
        bw = ws_inb[ind]
        bh = hs_inb[ind]

        box = [bcx/w, bcy/h, bw/w, bh/h]

        iou = bbox_iou(box, label, x1y1x2y2=False)
        print (iou)
        
        pred_infos.append([imgpath,box])
        proposals = proposals + iou
        total = total+1

0.0
0.0
0.586751405084
0.648609279152
0.0
0.729716846209
0.790333130822
0.0
0.0
0.726855127749
0.485746682508
0.763437563599
0.709402900521
0.0
0.465337456754
0.374943396648
0.0
0.0
0.706602283654
0.447532426714
0.0
0.46208895677
0.557894340541
0.611022630514
0.522004533858
0.148697317715
0.654387027572
0.786600404302
0.0
0.0
0.0
0.821006002505
0.516791521948
0.0
0.0
0.784711352785
0.0
0.809474299829
0.0489317717426
0.0
0.695279807355
0.0
0.371345476525
0.0
0.0855455881237
0.0
0.467609988444
0.0
0.430543016841
0.457038350658
0.678923419239
0.694189203491
0.786879874638
0.663458828319
0.248276475678
0.638945185994
0.613406858553
0.805635943209
0.877148220414
0.482308855772
0.319044181409
0.696525736478
0.545868566102
0.587184408345
0.559829259953
0.620437123757
0.0
0.578496596266
0.762276368929
0.841025488648
0.815095075051
0.831196863493
0.370816880896
0.500992690181
0.682745676915
0.266410120315
0.746823088687
0.0
0.894569654048
0.696489209491
0.657037875577
0.0
0.868747048896
0.69376

0.656543083878
0.0
0.717411300508
0.843359471582
0.300615746303
0.0
0.656953483393
0.237646962448
0.0
0.823365109644
0.633692991335
0.888947187362
0.545488330017
0.721596378714
0.556831852795
0.844785806958
0.535919471196
0.726672203761
0.589746773894
0.769678346577
0.399814993142
0.600162207146
0.55041723813
0.758776514589
0.801502256217
0.640173487524
0.0
0.459354720379
0.74625234239
0.883232215037
0.671005396412
0.72725257641
0.0
0.302012743143
0.819718670613
0.248175510721
0.0
0.46162971124
0.746835392564
0.0
0.662153931576
0.811792326085
0.0
0.806366168063
0.0
0.822106141087
0.0
0.0
0.71937644076
0.572588473415
0.0
0.830277726549
0.590032073939
0.540219542438
0.698538465013
0.23198834682
0.803427943976
0.579022585336
0.77906145051
0.666743550298
0.603798448416
0.444475565587
0.768152787645
0.717031346619
0.848575494171
0.74917725201
0.872749642827
0.53948232872
0.0551312556655
0.0
0.645040019013
0.0
0.414200364562
0.68113245476
0.66268693423
0.75078001011
0.611491142269
0.68126865

In [32]:
proposals/total

0.45871142181457863

In [35]:
with open('pred_infos.json', 'w') as outfile:
    json.dump(pred_infos, outfile)

In [36]:
param_list=[]
for key, value in model.state_dict().items():
    flat_weight = value.contiguous().view(value.numel())
    param_list.extend(flat_weight.tolist())
for i in param_list:
    i = float(i)

In [37]:
len(param_list)

48414

In [38]:
import struct
fp = open("param_v19best.bin",'wb')
s = struct.pack('f'*len(param_list), *param_list)
fp.write(s)

193656

In [40]:
img.size

(320, 176)